In [1]:
import pandas as pd
import plotly.graph_objects as go
from concurrent.futures import ThreadPoolExecutor
from taipy.gui import Gui, notify, invoke_long_callback
import taipy.gui.builder as tgb
import yfinance_cache as yfc

YFC upgrading cached calendars ...
YFC recalculating 'Final?' column in prices, estimate 0.0 minutes to process 2 tickers.


100%|██████████| 2/2 [00:00<00:00, 1002.22it/s]


In [2]:
# Get S&P 500 companies with theirs tickers: less stable but faster method
wiki_url = "https://en.wikipedia.org/wiki/List_of_S&P_500_companies"
# identify the table in the HTML by its unique id
sp500 = pd.read_html(wiki_url, attrs={"id": "constituents"})[0]
sp500.sort_values("Symbol", inplace=True)

In [3]:
ticker_list = [
    "GOOG",
    "TSLA",
    "AAPL",
    "BF.B",
    "META",
    "BRK.B",
]
start = "2024-01-01"
end = pd.Timestamp.today()
interval = "1d"

[ThreadPoolExecutor: the complete guide ](https://superfastpython.com/threadpoolexecutor-in-python/#Use_submit_with_as_completed])<br>
[yfinance ThreadPoolExecutor example](https://stackoverflow.com/questions/69983379/python-how-to-implement-concurrent-futures-to-a-function)<br>
[Stock fetching with ThreadPoolExecutor example](https://github.com/devfinwiz/Stock_Screeners_Raw/blob/master/Scipts/FinancialsExtractor.py)<br>
[yfinance ThreadPoolExecutor example](https://www.youtube.com/watch?v=wtOAh9KE0Ks)


In [43]:
def get_stocks_data(ticker_list, start, end, interval):
    ticker_modified_list = []
    for ticker in ticker_list:
        try:
            yfc.Ticker(ticker).info["shortName"]
            ticker_modified_list.append(ticker)
        except (KeyError, Exception):
            ticker_modified = ticker.replace(".", "-")
            ticker_modified_list.append(ticker_modified)

    def get_stock_data(ticker):
        stock_history = yfc.download(
            tickers=ticker,
            start=start,
            end=end,
            interval=interval,
            max_age=pd.Timedelta(days=1),
            threads=False,
        )
        stock_history = stock_history["Close"]
        return stock_history

    with ThreadPoolExecutor() as executor:
        fetched_data = pd.DataFrame()
        for ticker in ticker_modified_list:
            future = executor.submit(get_stock_data, ticker)
            fetched_data[ticker] = future.result()
    fetched_data.columns = ticker_list
    return fetched_data

c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\gevent\hub.py:161: UserWarning:

libuv only supports millisecond timer resolution; all times less will be set to 1 ms



```python
# 1. Reindex to include new dates
            all_dates = cache[ticker].index.union(fetched_data[ticker].index)
            cache[ticker] = cache[ticker].reindex(all_dates)
            # 2. Combine with fetched data, prioritizing existing values
            cache[ticker] = cache[ticker].combine_first(fetched_data[ticker])
```


In [26]:
stocks_data = get_stocks_data(ticker_list, start, end + pd.DateOffset(1), interval)

$BRK.B: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


- info:
{'maxAge': 86400, 'priceHint': 2, 'quoteType': 'EQUITY', 'symbol': 'BRK.B', 'underlyingSymbol': 'BRK.B', 'uuid': '5a2c1675-19de-345f-a8c3-a61c958d64f1', 'trailingPegRatio': None}


100%|██████████| 1/1 [00:00<00:00, 43.59it/s]


In [39]:
def get_stocks_data_status(state, status, result):
    if status:
        state.stocks_data = result
        notify(state, "success", "Historical data has been updated")
        state.refresh("stocks_data")
        # state.refresh("create_cards")
        # state.refresh("create_line_chart")
    else:
        notify(state, "error", "Failed to update historical data")

c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\gevent\hub.py:161: UserWarning:

libuv only supports millisecond timer resolution; all times less will be set to 1 ms



In [44]:
gui.reload()

[2025-01-28 03:45:06.321][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2025-01-28 03:45:08.390][Taipy][INFO]  * Server starting on http://127.0.0.1:5000
[2025-01-28 03:45:08.417][Taipy][INFO] Gui server has been reloaded.


c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\gevent\hub.py:161: UserWarning:

libuv only supports millisecond timer resolution; all times less will be set to 1 ms



{'action': 'update_date_range', 'autosize': True, 'args': []}


c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\taipy\gui\gui.py:852: TaipyGuiWarning:

update_charts(): callback function raised an exception:
0

$BRK.B: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


- info:
{'maxAge': 86400, 'priceHint': 2, 'quoteType': 'EQUITY', 'symbol': 'BRK.B', 'underlyingSymbol': 'BRK.B', 'uuid': '5a2c1675-19de-345f-a8c3-a61c958d64f1', 'trailingPegRatio': None}


...

  0%|          | 0/1 [00:00<?, ?it/s]
c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\taipy\gui\gui_actions.py:390: TaipyGuiWarning:

invoke_long_callback(): Exception raised in function get_stocks_data():
'interval' if str must be one of: dict_keys(['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '1wk'])

c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\taipy\gui\gui.py:852: TaipyGuiWarning:

update_charts(): callback function raised an exception:
0

$BRK.B: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


- info:
{'maxAge': 86400, 'priceHint': 2, 'quoteType': 'EQUITY', 'symbol': 'BRK.B', 'underlyingSymbol': 'BRK.B', 'uuid': '5a2c1675-19de-345f-a8c3-a61c958d64f1', 'trailingPegRatio': None}


100%|██████████| 1/1 [00:00<00:00, 22.32it/s]
c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\taipy\gui\gui.py:852: TaipyGuiWarning:

update_charts(): callback function raised an exception:
0

$BRK.B: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


- info:
{'maxAge': 86400, 'priceHint': 2, 'quoteType': 'EQUITY', 'symbol': 'BRK.B', 'underlyingSymbol': 'BRK.B', 'uuid': '5a2c1675-19de-345f-a8c3-a61c958d64f1', 'trailingPegRatio': None}


  0%|          | 0/1 [00:00<?, ?it/s]
c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\taipy\gui\gui_actions.py:390: TaipyGuiWarning:

invoke_long_callback(): Exception raised in function get_stocks_data():
'interval' if str must be one of: dict_keys(['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '1wk'])



In [29]:
def update_charts(state):
    notify(state, "info", "Fetching data")
    invoke_long_callback(
        state,
        get_stocks_data,
        [state.ticker_list, state.start, state.end, state.interval],
        get_stocks_data_status,
    )
    # state.stocks_data = get_stocks_data(state.ticker_list, state.start, state.end, state.interval)
    # state.refresh("stocks_data")
    # Notify when no data found:
    if len(state.stocks_data[0]) == 0:
        notify(
            state,
            "error",
            f"Error: No data found for {state.ticker} from {state.start} to {state.end}",
        )

In [30]:
start_range = None
end_range = None


def update_date_range(state, id, payload):
    print(payload)
    state.start_range = payload.get("xaxis.range[0]")
    state.end_range = payload.get("xaxis.range[1]")
    # Alternative: try-except KeyError
    # state.date_range = payload["xaxis.range[0]"] doesn't work. Reason: keys in the payload might vary depending on the type of interaction that triggered the callback. For example, if simply clicks on the chart without changing the range, the payload might not contain the "xaxis.range[0]" key. Whereas payload.get("xaxis.range[0]"): safely retrieves the value associated with the key "xaxis.range[0]" and if the key is not found, it returns None by default (or a specified default value).

In [31]:
def create_cards(ticker_list, stocks_data, start_range, end_range):
    # Dynamically calculate plotly subplot grid layout
    n_plots = len(ticker_list)
    # Square root aims to create a balanced grid with roughly equal numbers of rows and columns:
    cols = 4 if n_plots < 16 else int(n_plots ** (1 / 2))
    # Round up by double negative of result from rounding down:
    rows = -(-n_plots // cols)
    # Substract the top and bottom margins in proportion to total height
    available_space = 1 - (55 + 10) / (120 * rows)
    # Add 0.5 to account for the space above the 1st row and below the last row
    row_spacing = available_space / rows
    fig_sparkline = go.Figure().set_subplots(
        rows, cols, horizontal_spacing=0.1, vertical_spacing=row_spacing
    )
    fig_sparkline.update_layout(
        margin={"l": 100, "r": 30, "t": 55, "b": 10},
        height=120 * rows,
        hoverlabel_align="right",
    )
    fig_sparkline.update_xaxes(showgrid=False, visible=False)
    fig_sparkline.update_yaxes(showgrid=False, visible=False)
    for i, ticker in enumerate(ticker_list):
        # stocks_data[ticker] = stocks_data[ticker][date_range:] only works twice
        row = (i // cols) + 1  # round down to whole nearest number
        col = (i % cols) + 1  # division remainder
        fig_sparkline.add_trace(
            go.Scatter(
                x=stocks_data.loc[start_range:end_range, ticker].index,
                y=stocks_data.loc[start_range:end_range, ticker],
                fill="tozeroy",
                line_color="red",
                fillcolor="pink",
                showlegend=False,
                name=ticker,
                hovertemplate="%{x|%d/%m/%Y}: <b>%{y:$.2f}</b>",
            ),
            row=row,
            col=col,
        )
        # Calculate delta for annotations:
        delta_percent = (
            stocks_data[ticker].iloc[-1] / stocks_data[ticker].iloc[-2]
        ) - 1
        delta_symbol = "▲" if delta_percent >= 0 else "▼"
        delta_color = "green" if delta_percent >= 0 else "red"
        # Insert annotations:
        fig_sparkline.add_annotation(
            text=f"{ticker}<br><span style='color:{delta_color}'>{delta_symbol} {abs(delta_percent):.2%}</span>",
            xref="x domain",  # Refer to the x-axis domain of the subplot
            yref="y domain",  # Refer to the y-axis domain of the subplot
            x=1,  # Position 100% from the left (almost right edge)
            y=1.7,  # Position 115% from the bottom (almost top edge)
            row=row,
            col=col,
            showarrow=False,
            align="right",
        )
        fig_sparkline.add_annotation(
            text=f"<b>{sp500.loc[sp500["Symbol"]==ticker,"Security"].iloc[0]}</b><br><br><span style='color:grey'>Last Price</span><br><b>${stocks_data[ticker].iloc[-1]:,.2f}</b>",
            xref="x domain",
            yref="y domain",
            x=-0.4,
            y=1.7,
            row=row,
            col=col,
            showarrow=False,
            align="left",
        )
        # Insert rounded-corner borders using SVG `path` syntax:
        x0, y0 = -0.4, -0.08
        x1, y1 = 1.02, 1.8
        radius = 0.07
        rounded_bottom_left = f" M {x0+radius}, {y0} Q {x0}, {y0} {x0}, {y0+radius}"
        rounded_top_left = f" L {x0}, {y1-radius} Q {x0}, {y1} {x0+radius}, {y1}"
        rounded_top_right = f" L {x1-radius}, {y1} Q {x1}, {y1} {x1}, {y1-radius}"
        rounded_bottom_right = f" L {x1}, {y0+radius} Q {x1}, {y0} {x1-radius}, {y0}Z"
        path = (
            rounded_bottom_left
            + rounded_top_left
            + rounded_top_right
            + rounded_bottom_right
        )
        fig_sparkline.add_shape(
            type="path",
            path=path,
            xref="x domain",
            yref="y domain",
            row=row,
            col=col,
            line={"color": "grey"},
        )
    return fig_sparkline

In [32]:
create_cards(ticker_list, stocks_data, start_range, end_range)

In [33]:
def create_line_chart(ticker_list, stocks_data):
    fig_line_chart = go.Figure()
    for ticker in ticker_list:
        fig_line_chart.add_trace(
            go.Scatter(
                x=stocks_data[ticker].index,
                y=stocks_data[ticker],
                name=ticker,
                showlegend=True,
                hovertemplate="%{x|%d/%m/%Y}: <b>%{y:$,.2f}</b>",
            )
        )
    fig_line_chart.update_xaxes(
        rangeselector={
            "buttons": [
                {
                    "label": "1 month",
                    "count": 1,
                    "step": "month",
                    "stepmode": "backward",
                },
                {
                    "label": "6 months",
                    "count": 6,
                    "step": "month",
                    "stepmode": "backward",
                },
                {"label": "YTD", "count": 1, "step": "year", "stepmode": "todate"},
                {"label": "1 year", "count": 1, "step": "year", "stepmode": "backward"},
                {"step": "all"},
            ],
            "bgcolor": "rgba(0,0,0,0)",
            "activecolor": "gray",
            "bordercolor": "gray",
            "borderwidth": 1,
        }
    )
    fig_line_chart.update_layout(
        title={
            "text": f"<b>Historical Price over the Period for</b>: {", ".join(ticker_list)}",
            "y": 0.96,
        },
        yaxis={
            "title": "<b>US$</b>",
            "fixedrange": False,  # If True, then zoom is disabled
        },
        margin_pad=10,  # space between tick labels & graph
        margin={"b": 30, "t": 80},
        hoverlabel_align="right",
    )
    return fig_line_chart

In [34]:
create_line_chart(ticker_list, stocks_data)

In [35]:
company_list = list(zip(sp500["Symbol"], sp500["Symbol"] + ": " + sp500["Security"]))
interval_list = [  # 1 minute is available but date range would be limited to 8 days
    ("1d", "1 day"),
    ("5d", "5 days"),
    ("1wk", "1 week"),
    ("1mo", "1 month"),
    ("3mo", "3 months"),
]
with tgb.Page() as page:
    with tgb.part("container"):
        with tgb.layout(columns="1 2", gap="30px", class_name="card"):
            with tgb.part():
                tgb.text("#### Selected **Period**", mode="md")
                tgb.text("**From:**", mode="md")
                tgb.date(
                    "{start}",
                    format="dd/MM/y",
                    on_change=update_charts,
                )
                tgb.text("**To:**", mode="md")
                tgb.date(
                    "{end}",
                    format="dd/MM/y",
                    on_change=update_charts,
                )
            with tgb.part():
                tgb.text("#### Selected **Ticker**", mode="md")
                tgb.text("Choose any tickers from the dropdown list below:", mode="md")
                tgb.selector(
                    value="{ticker_list}",
                    label="Companies",
                    dropdown=True,
                    multiple=True,
                    lov="{company_list}",  # search-in-place or search-within-dropdown
                    on_change=update_charts,
                    value_by_id=True,
                )
                tgb.text("Choose interval:", mode="md")
                tgb.toggle(
                    value="{interval}",
                    lov="{interval_list}",
                    on_change=update_charts,
                    value_by_id=True,
                )
        tgb.html("br")
        tgb.chart(
            figure="{create_cards(ticker_list,stocks_data,start_range,end_range)}"
        )
        tgb.html("br")
        tgb.chart(
            figure="{create_line_chart(ticker_list,stocks_data)}",
            on_range_change=update_date_range,
        )

In [36]:
gui = Gui(page)
gui.run(dev_mode=True, watermark="")

[2025-01-28 03:33:49.331][Taipy][INFO] Running in 'single_client' mode in notebook environment
[2025-01-28 03:33:53.981][Taipy][INFO]  * Server starting on http://127.0.0.1:5000


c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\gevent\hub.py:161: UserWarning:

libuv only supports millisecond timer resolution; all times less will be set to 1 ms



{'action': 'update_date_range', 'autosize': True, 'args': []}
{'action': 'update_date_range', 'xaxis.range[0]': '2024-01-27', 'xaxis.range[1]': '2025-01-27', 'args': []}
{'action': 'update_date_range', 'xaxis.autorange': True, 'args': []}


c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\taipy\gui\gui.py:852: TaipyGuiWarning:

update_charts(): callback function raised an exception:
0

$BRK.B: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


- info:
{'maxAge': 86400, 'priceHint': 2, 'quoteType': 'EQUITY', 'symbol': 'BRK.B', 'underlyingSymbol': 'BRK.B', 'uuid': '5a2c1675-19de-345f-a8c3-a61c958d64f1', 'trailingPegRatio': None}


...

100%|██████████| 1/1 [00:00<00:00, 18.92it/s]
c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\taipy\gui\gui.py:852: TaipyGuiWarning:

update_charts(): callback function raised an exception:
0

$BRK.B: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


- info:
{'maxAge': 86400, 'priceHint': 2, 'quoteType': 'EQUITY', 'symbol': 'BRK.B', 'underlyingSymbol': 'BRK.B', 'uuid': '5a2c1675-19de-345f-a8c3-a61c958d64f1', 'trailingPegRatio': None}


100%|██████████| 1/1 [00:00<00:00, 26.24it/s]
c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\taipy\gui\utils\_evaluator.py:369: TaipyGuiWarning:

Exception raised evaluating create_cards(ticker_list,stocks_data,start_range,end_range):
'TSCO'

c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\taipy\gui\utils\_evaluator.py:369: TaipyGuiWarning:

Exception raised evaluating create_line_chart(ticker_list,stocks_data):
'TSCO'

c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\taipy\gui\gui.py:852: TaipyGuiWarning:

update_charts(): callback function raised an exception:
0

$BRK.B: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


- info:
{'maxAge': 86400, 'priceHint': 2, 'quoteType': 'EQUITY', 'symbol': 'BRK.B', 'underlyingSymbol': 'BRK.B', 'uuid': '5a2c1675-19de-345f-a8c3-a61c958d64f1', 'trailingPegRatio': None}


100%|██████████| 1/1 [00:00<00:00,  1.94it/s]


In [38]:
gui.state.stocks_data

,GOOG,TSLA,AAPL,BF.B,META,BRK.B
Date,,,,,,
2024-01-03 00:00:00-05:00,139.857483,238.449997,183.351746,54.738899,343.159180,366.750000
2024-01-04 00:00:00-05:00,137.545776,237.929993,181.023163,54.807606,345.799072,363.679993
2024-01-05 00:00:00-05:00,136.898117,237.490005,180.296707,54.071472,350.610687,365.589996
2024-01-08 00:00:00-05:00,140.026855,240.449997,184.655365,54.660381,357.295135,368.179993
2024-01-09 00:00:00-05:00,142.049606,234.960007,184.237411,54.532787,356.069824,366.899994
...,...,...,...,...,...,...
2025-01-21 00:00:00-05:00,199.630005,424.070007,222.639999,33.799999,616.460022,468.570007
2025-01-22 00:00:00-05:00,200.029999,415.109985,223.830002,33.029999,623.500000,460.510010
2025-01-23 00:00:00-05:00,199.580002,412.380005,223.660004,33.150002,636.450012,459.829987


c:\Users\maitp\anaconda3\envs\general\Lib\site-packages\gevent\hub.py:161: UserWarning:

libuv only supports millisecond timer resolution; all times less will be set to 1 ms

